# II. Min hashing

In [3]:
import pandas as pd
import numpy as np
import collections
import itertools
import random
from tqdm import tqdm
from math import floor, sqrt
import ast

Cargar datos preprocesados

In [4]:
resumenx = pd.read_csv('preprocesado.csv', converters={"shingles": ast.literal_eval})

Cargar vocabulario de shingles previamente computado y mostrar su largo

In [5]:
with open('shingles_sample.txt','r') as f:
   shingles_sample = ast.literal_eval(f.read())
un = shingles_sample
len(un)

9958

Esta parte simplemente la usé para debuggear, pero no borrar

In [6]:
resumen = resumenx
resumen

,Unnamed: 0,id,screen_name,text,clean_text,shingles
0,7,1512188055788027904,Rob92029357,@sokio @berfontaine No tienes como revatir alg...,@sokio @berfontaine revatir realidad expuesta ...,"(sabe , ontai, pers, ad ex, kio @, ona s, rev..."
1,20,1512193611575799812,AlejoLatigo,@Hugo_Gutierrez_ Yaaaaaa yyyyyyyyy????????????...,@Hugo_Gutierrez_ Yaaaaaa yyyyyyyyy????????????...,"(? #RE, ?? #R, @Ser, ??? #, yy???, aaa y, yyy..."
2,29,1512198930657955842,GordoUC_,@Molines01 @tere_marinovic Te están mostrando ...,@Molines01 @tere_marinovic estn mostrando rech...,"(echa., nes01, den e, o rec, mostr, oline, ari..."
3,30,1512199583794835467,Orellana21Marce,@mdaza_abogado @Jess99041498 Es que la derecha...,@mdaza_abogado @Jess99041498 derecha quiere em...,"(o @Je, @Jes, impo, quie, 8 der, n ind, @md..."
4,50,1512212224613359617,PATRIOTCHILENO,@tere_marinovic @Sebasti45781359 Orsoni cero a...,@tere_marinovic @Sebasti45781359 Orsoni cero a...,"(45781, as to, @Mait, 81359, ere_m, Maite, ovi..."
...,...,...,...,...,...,...
203721,1266590,1526783486702411776,MaraTer03135126,@BSepulvedaHales aparacistes loca populista #R...,@BSepulvedaHales aparacistes loca populista #R...,"(hazoE, s apa, ista , co/JL, ho ht, Mamar, o h..."
203722,1266611,1527138136060334080,cuncunga1,@mcubillossigall Aqui no hubo bandera ni canci...,@mcubillossigall Aqui bandera cancin nacional....,"(queda, cubil, l. qu, l Aqu, Aqui, http, lad..."
203723,1266615,1527090757202456576,Erwinpohlagurto,@Hugo_Gutierrez_ Últimamente sospecho de todos...,"@Hugo_Gutierrez_ ltimamente sospecho todos, am...","(ues e, http, ltima, amigu, UKXHt, os ht, se..."
203724,1266621,1525908559673712640,RichardDCorobo,@BottoConstituy1 El poder económico nunca va a...,@BottoConstituy1 poder econmico nunca va quere...,"(nca v, co nu, y1 po, http, LkfWL, uerer, ca ..."


## Crear funciones de hash

Primero hallamos un número primo más grande que el universo de shingles

In [7]:
def is_prime(n):
  for i in range(2,int(sqrt(n))+1):
    if (n%i) == 0:
      return False
  return True

numb = len(un)+1
while True:
    result = is_prime(numb)
    if result:
       break
    numb+=1
print("Primo encontrado: ", numb)

Primo encontrado:  9967


Creamos 20 funciones de hash

In [8]:
def crear_hash(a, b, p, n):
    def f(x):
        return ((a * x + b) % p) % n
    return f

In [9]:
h = []
n = len(un)
p = numb
num_hash = 20
for i in range(num_hash):
    a = random.randint(1,p-1)
    b = random.randint(1,p-1)
    h.append(crear_hash(a,b,p,n))

## Firma de Hash

Calcularemos la firma de hash dividiendo el resumen en 100 chunks.

In [10]:
sorted_union = sorted(un)
def chunk_signature(chunk):
    n = len(chunk)
    FH = np.full((num_hash, n), n+100)
    ### Iterar por elementos del vocabulario
    for e in tqdm(range(0,len(sorted_union))): # |union_shingles| = 9967
       
        ### iteramos cada columna, correspondiente a la cantidad de textos (tweets en nuestro caso)  
        for i in range(0,n): # |cols| = 208.139

            ### si el elemento e no está en el texto i, no hacemos nada
            if sorted_union[e] not in chunk.shingles[i]:
                pass
            
            ### si el elemento si está, computamos los valores de h(e) y 
            else:
                for j in range(0,num_hash): # j es función de hash |hash| = 20, filas de la matriz
                    FH[j][i] = min(FH[j][i],h[j](e))
    return FH

Calcular la firma de hash de cada chunk e ir escribiendo a disco

In [11]:
# n_of_chunks = 10
# chunk_size = floor(len(resumen)/n_of_chunks)
# chunk = resumen.iloc[:chunk_size,:]
# FH = chunk_signature(chunk)
# np.savetxt("output/file0.csv", FH, delimiter=",")
# del FH
# for i in tqdm(range(1, n_of_chunks)):
#     if i != (n_of_chunks-1):
#         chunk = resumen.iloc[chunk_size*i:chunk_size*(i+1),:]
#     else:
#         chunk = resumen.iloc[chunk_size*i:,:]
#     chunk = chunk.reset_index()
#     FH = chunk_signature(chunk)
#     np.savetxt(f"output/file{i}.csv", FH, delimiter=",")
#     del FH

Consolidar FHs a partir de los archivos guardados en disco.

In [12]:
# FH = np.genfromtxt('output/file0.csv', delimiter=',')
# FH2 = np.genfromtxt('output/file1.csv', delimiter=',')
# # FH.shape, FH

In [15]:
FH = np.genfromtxt('output/file0.csv', delimiter=',')
for i in range(1, 10):
    FH2 = np.genfromtxt(f'output/file{i}.csv', delimiter=',')
    FH = np.concatenate((FH, FH2), axis=1)
FH.shape

(20, 203726)

## Banding

Mostrar distancia mínima a la que debe estar dos tweet para ser considerados pares candidatos

In [16]:
r = 5
b = 4
(1/b)**(1/r)

0.757858283255199

In [18]:
# https://towardsdatascience.com/locality-sensitive-hashing-how-to-find-similar-items-in-a-large-set-with-precision-d907c52b05fc
sig_mat = FH
n, d = sig_mat.shape
#assert(n==b*r)
hashbuckets = collections.defaultdict(set)
bands = np.array_split(sig_mat, b, axis=0)
for i,band in enumerate(bands):
    for j in range(d):
        # The last value must be made a string, to prevent accidental
        # key collisions of r+1 integers when we really only want
        # keys of r integers plus a band index
        band_id = tuple(list(band[:,j])+[str(i)])
        hashbuckets[band_id].add(j)
candidate_pairs = set()
for bucket in hashbuckets.values():
    if len(bucket) > 1:
        for pair in itertools.combinations(bucket, 2):
            candidate_pairs.add(pair)
print(f"Obtuvimos {len(candidate_pairs)} pares candidatos, con b = {b} y r = {r}")

Obtuvimos 2235158 pares candidatos, con b = 4 y r = 5


Obtener pares candidatos

## Limpiar pares de candidatos

Quitar tweets idénticos de los pares similares

In [19]:
trimmed_cadidate_pairs = []
for pair in candidate_pairs:
        a, b = pair
        text_a = resumenx.text[a]
        text_b = resumenx.text[b]
        if (text_a != text_b) and ('http' not in text_a) and ('http' not in text_b):
                trimmed_cadidate_pairs.append((a, b))
print(f"Quedan {len(trimmed_cadidate_pairs)} pares candidatos")

Quedan 1035758 pares candidatos


Mostrar 3 pares de tweets presuntamente similares

In [20]:
for pair in random.sample(list(trimmed_cadidate_pairs), 3):
    a, b = pair
    print(f"1. {resumenx.screen_name[a]}: ",resumen.text[a])
    print(f"2. {resumenx.screen_name[b]}: ",resumen.text[b])
    print("----------")

1. SergioP88693823:  @patriciapolitz @convencioncl @sebastian_gray @lanetacl @mariomarcelc El borrador se va al Pluri-nario
2. Claudia43601160:  @patriciapolitz @convencioncl @sebastian_gray @lanetacl @mariomarcelc Igualito q Venezuela
----------
1. en_iquique:  @patriciapolitz @convencioncl @T_Pustilnick @dbravosilva @mdaza_abogado @GuillermoNamor Por fin se acaba esta pesadilla!!!!!
2. JosAnto67759035:  @patriciapolitz @convencioncl @NoNeutrales @mdaza_abogado Que decepcion esta señora .nunca pense tan resentida
----------
1. Chileno17039890:  @gdominguez_ @Mulenypunto3 @MEQChile @ElisaLoncon @Jaime_Bassa @convencioncl Qué imágen más terrorífica y dantesca
2. mbravogalaz:  @gdominguez_ @MEQChile @ElisaLoncon @Jaime_Bassa @convencioncl De puro mirar la foto me dieron 🤮🤮🤮🤮
#RechazoElMamarracho
----------


## Identificar autores similares

Contar cuantos tweets similares hay para cada par de autores considerados candidatos similares

In [21]:
count_dict = dict()
for pair in trimmed_cadidate_pairs:
    a, b = pair
    user_a = resumenx.screen_name[a]
    user_b = resumenx.screen_name[b]
    if user_a != user_b:
        try:
            count_dict[user_a][user_b] += 1
        except KeyError:
            try:
                count_dict[user_a] = dict()
                count_dict[user_a][user_b] += 1
            except KeyError:
                count_dict[user_a][user_b] = 1
        try:
            count_dict[user_b][user_a] += 1
        except KeyError:
            try:
                count_dict[user_b] = dict()
                count_dict[user_b][user_a] += 1
            except KeyError:
                count_dict[user_b][user_a] = 1

Mostrar solo los autores que están por sobre un threshold de tweets similares

In [22]:
count_df = pd.DataFrame(count_dict).reset_index()
count_long = pd.melt(count_df, id_vars='index')
similar_authors = count_long[count_long.value >= 3].rename({'index': 'user_1', 'variable': 'user_2'}, axis = 1)
similar_authors = similar_authors[similar_authors.value <= 30]
similar_authors

,user_1,user_2,value
2153843,salvador_3_68,enalvareli,3.0
2890499,salvador_3_68,pamelasexi18,18.0
2893171,pamelasexi18,salvador_3_68,5.0
3605273,Quenna4,Palomaenlibert2,5.0
5851797,RomilioValenzu4,soydelever69,3.0
8418034,Tol1Soto,marcelo_arrieta,12.0
9948202,Tol1Soto,0_sonero,5.0
10367328,A_Erazo_P,bmachuca2,3.0
10739388,mono_dex,e_constituyente,4.0


Función para imprimir tweets de dos candidatos pares:

In [23]:
def show_tweets_from_candidates(user_1, user_2):
    for pair in trimmed_cadidate_pairs:
        a, b = pair
        user_a = resumenx.screen_name[a]
        user_b = resumenx.screen_name[b]
        if ((user_a == user_1) and (user_b == user_2)) or ((user_b == user_1) and (user_a == user_2)):
            print("By user ", user_a,":\n",  f'"{resumenx.text[a]}"\n')
            print("By user ", user_b, ":\n", f'"{resumenx.text[b]}"')
            print("_______________________________________________________\n")

Mostrar pares de tweets de todos los autores similares

In [24]:
for index, row in similar_authors.iterrows():
    user_1 = row.user_1
    user_2 = row.user_2
    show_tweets_from_candidates(user_1, user_2)
    

By user  enalvareli :
 "#CongresoBasuraYGolpista 
#CierrenElCongreso 
#CierrenElCongreso #NuevaConstitucion #grupoelcomercio #Gestion"

By user  salvador_3_68 :
 "#CierreDelCongreso  #NuevaConstitucion 
#PrensaBasuraYGolpista 
Castillo Keiko Fujimori
Xi Jinping FIFA Metro"
_______________________________________________________

By user  enalvareli :
 "#CongresoBasuraYGolpista 
#CierrenElCongreso 
#CierrenElCongreso #NuevaConstitucion"

By user  salvador_3_68 :
 "#CierreDelCongreso  #NuevaConstitucion 
#PrensaBasuraYGolpista 
Sigrid María del Carmen Alva
Yotun Guerrero"
_______________________________________________________

By user  enalvareli :
 "#CongresoBasuraYGolpista 
#CierrenElCongreso 
#CierrenElCongreso #NuevaConstitucion"

By user  salvador_3_68 :
 "#CierrenElCongreso  #NuevaConstitucion 
#PrensaBasuraYGolpista 
Willax limaflores Bruno Pacheco"
_______________________________________________________

By user  salvador_3_68 :
 "#CierreDelCongreso  #NuevaConstitucion 
#PrensaB

KeyboardInterrupt: 